In [39]:
from keys import access_key, secret_key
import jwt  # PyJWT
import uuid
import websocket  # websocket-client
import requests
import datetime
import time
import pandas as pd

In [2]:
# Authentication Progress

def on_message(ws, message):
    # do something
    data = message.decode('utf-8')
    print(data)


def on_connect(ws):
    print("connected!")
    # Request after connection
    ws.send('[{"ticket":"test example"},{"type":"myTrade"}]')


def on_error(ws, err):
    print(err)


def on_close(ws, status_code, msg):
    print("closed!")


payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
}

jwt_token = jwt.encode(payload, secret_key)
authorization_token = 'Bearer {}'.format(jwt_token)
headers = {"Authorization": authorization_token}

ws_app = websocket.WebSocketApp("wss://api.upbit.com/websocket/v1",
                                header=headers,
                                on_message=on_message,
                                on_open=on_connect,
                                on_error=on_error,
                                on_close=on_close)
ws_app.run_forever()

connected!

closed!


True

In [32]:
# current_second = datetime.now().second
# current_minute = datetime.now().minute
# current_hour = datetime.now().hour

# current_day = datetime.now().day
# current_month = datetime.now().month
# current_year = datetime.now().year

url = "https://api.upbit.com/v1/candles/minutes/1?market=KRW-BTC&to=2024-07-15T01%3A00%3A00&count=20"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.json()[1])

{'market': 'KRW-BTC', 'candle_date_time_utc': '2024-07-15T00:58:00', 'candle_date_time_kst': '2024-07-15T09:58:00', 'opening_price': 86216000.0, 'high_price': 86278000.0, 'low_price': 86216000.0, 'trade_price': 86233000.0, 'timestamp': 1721005140037, 'candle_acc_trade_price': 182812908.72689, 'candle_acc_trade_volume': 2.11990367, 'unit': 1}


In [36]:
def get_minute_BIT(market, year, month, day, hour, minute, second):
	dataset = []
	url = "https://api.upbit.com/v1/candles/minutes/1"
	to_utc = datetime.datetime(year, month, day, hour, minute, second)
	to_kst = to_utc + datetime.timedelta(hours=9)
	to_kst_str = to_kst.strftime("%Y-%m-%dT%H:%M:%S")
	queryparams = {
		"market": market,
		"to": f"{to_kst_str}",
		"count": 144
    }
	headers = {"accept": "application/json"}
	res = requests.get(url, headers=headers, params=queryparams)
	# 요청 성공 했을 경우 : code 200
	if res.status_code == 200:
		try:
			for i in range(144):
				data = res.json()[i]
				# 필요 데이터
				candle_date_time_kst = data["candle_date_time_kst"] 
				close_price = data["trade_price"] # 종가
				candle_acc_trade_volume = data["candle_acc_trade_volume"] #누적 거래량
				dataset.append([candle_date_time_kst, close_price, candle_acc_trade_volume])
			return dataset
			
		except Exception as e:
			raise Exception(e)
	else:
		res.status_code == 400

In [37]:
# 종목 = 비트코인(KRW)
market = "KRW-BTC"

# []에 KRW-BTC 분 단위 데이터 저장
minute_data = []

start_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 6, 30, 23, 59)

current_date = start_date
while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    hour = current_date.hour
    minute = current_date.minute
    second = current_date.second
    
    try:
        data = get_minute_BIT(market, year, month, day, hour, minute, second)
        minute_data += data
    except Exception as e:
        print(f"Failed to fetch data for {current_date}: {e}")
    
    current_date += datetime.timedelta(minutes=144)
    time.sleep(0.125)

In [42]:
df = pd.DataFrame(minute_data,columns=['time','close','volume'])
df


,time,close,volume
0,2024-01-01T17:59:00,57267000.0,1.481283
1,2024-01-01T17:58:00,57269000.0,0.837176
2,2024-01-01T17:57:00,57269000.0,1.075063
3,2024-01-01T17:56:00,57272000.0,1.043744
4,2024-01-01T17:55:00,57274000.0,1.637384
...,...,...,...
262075,2024-07-01T13:16:00,88900000.0,2.099068
262076,2024-07-01T13:15:00,88903000.0,1.150951
262077,2024-07-01T13:14:00,88930000.0,0.424065
262078,2024-07-01T13:13:00,88931000.0,1.263964


In [41]:
df.tail()

,time,close,volumne
262075,2024-07-01T13:16:00,88900000.0,2.099068
262076,2024-07-01T13:15:00,88903000.0,1.150951
262077,2024-07-01T13:14:00,88930000.0,0.424065
262078,2024-07-01T13:13:00,88931000.0,1.263964
262079,2024-07-01T13:12:00,88950000.0,0.413937
